In [1]:
import mitsuba as mi
import drjit as dr
import numpy as np
import platform
import matplotlib.pyplot as plt

if platform.system() == 'Darwin':
    mi.set_variant('llvm_acoustic')
else:
    mi.set_variant('cuda_acoustic')

print(f'Mitsuba variant: {mi.variant()}')

Mitsuba variant: llvm_acoustic


In [2]:
spectrum = mi.load_dict({
    "type": "acoustic",
    "frequencies": "200, 300, 500",
    "values": "0.5, 2, 1"
})
print(spectrum)

2023-10-24 13:40:40 DEBUG main [PluginManager] Loading plugin "plugins/acoustic.dylib" ..

AcousticSpectrum[
m_distr = IrregularContinuousDistribution[
    size = 3,
    nodes = [200, 300, 500],
    integral = [425],
    pdf = [0.5, 2, 1],
  ]
]
m_distr_log = IrregularContinuousDistribution[
    size = 3,
    nodes = [7.64386, 8.22882, 8.96578],
    integral = [1.83665],
    pdf = [0.5, 2, 1],
  ]
m_distr_wavelengths = IrregularContinuousDistribution[
    size = 3,
    nodes = [0.68, 1.13333, 1.7],
    integral = [1.38833],
    pdf = [1, 2, 0.5],
  ]



In [3]:
si = mi.SurfaceInteraction3f()

# note that the distribution is sorted by wavelengths in ascending order
frequencies = [100, 200, 300, 400  , 500, 600]
values      = [0  , 0.5, 2  , 1.375, 1  , 0  ] # at 400 Hz the value is interpolated in wavelength space
c = 340
for i in range(len(values)):
    si.wavelengths = c / frequencies[i]
    print(f'frequency: {frequencies[i]} Hz')
    print(f'    wavelength: {si.wavelengths}')
    print(f'    values[i]: {values[i]}')
    print(f'    eval(): {spectrum.eval(si)}')
    # assert dr.allclose(spectrum.eval(si), values[i])

    print(f'    PDF: {spectrum.pdf_spectrum(si)}')
    print(f'    values[i]/integral: {values[i] / 1.38833}\n') # divide by integal of pdf
    # assert dr.allclose(spectrum.pdf_spectrum(si), values[i] / 1.38833)

frequency: 100 Hz
    wavelength: [[3.4000000953674316]]
    values[i]: 0
    eval(): [[0.0]]
    PDF: [[0.0]]
    values[i]/integral: 0.0

frequency: 200 Hz
    wavelength: [[1.7000000476837158]]
    values[i]: 0.5
    eval(): [[0.5]]
    PDF: [[0.3601440489292145]]
    values[i]/integral: 0.3601449223167402

frequency: 300 Hz
    wavelength: [[1.1333333253860474]]
    values[i]: 2
    eval(): [[2.0]]
    PDF: [[1.440576195716858]]
    values[i]/integral: 1.440579689266961

frequency: 400 Hz
    wavelength: [[0.8500000238418579]]
    values[i]: 1.375
    eval(): [[1.375]]
    PDF: [[0.9903961420059204]]
    values[i]/integral: 0.9903985363710357

frequency: 500 Hz
    wavelength: [[0.6800000071525574]]
    values[i]: 1
    eval(): [[1.0]]
    PDF: [[0.720288097858429]]
    values[i]/integral: 0.7202898446334804

frequency: 600 Hz
    wavelength: [[0.5666666626930237]]
    values[i]: 0
    eval(): [[0.0]]
    PDF: [[0.0]]
    values[i]/integral: 0.0



# Sampling of a uniform spectrum

In order to sample a uniform spectrum, we need to sample in logarithmic frequency space so that importance sampling produces equal amounts of rays in each frequency band. 

The following code shows how to sample a uniform spectrum using the `sample_spectrum()` method of the `mitsuba.Spectrum` class:

In [4]:
spectrum = mi.load_dict({
    "type": "acoustic",
    "frequencies": "250, 1000",
    "values": "1, 1"
})
print(spectrum)

AcousticSpectrum[
m_distr = IrregularContinuousDistribution[
    size = 2,
    nodes = [250, 1000],
    integral = [750],
    pdf = [1, 1],
  ]
]
m_distr_log = IrregularContinuousDistribution[
    size = 2,
    nodes = [7.96578, 9.96578],
    integral = [2],
    pdf = [1, 1],
  ]
m_distr_wavelengths = IrregularContinuousDistribution[
    size = 2,
    nodes = [0.34, 1.36],
    integral = [1.02],
    pdf = [1, 1],
  ]



In [5]:
print(f'For uniform samples s=0 and s=1, the spectrum is sampled at the min and max wavelength, respectively, which corresponds to the min and max frequency of 250 Hz and 1000 Hz.')
print('For s=0.5, the sample is at the middle of the spectrum in logarithmic frequency space (500 Hz).\n')

for s in [0, .5, 1]:
    # Sample the spectrum using the current uniform sample
    sample = spectrum.sample_spectrum(si, s)

    print(f'uniform sample: {s}')
    print(f'    wavelength sample: {sample[0]}')
    print(f'    frequency of sample: {c/sample[0]}')

For uniform samples s=0 and s=1, the spectrum is sampled at the min and max wavelength, respectively, which corresponds to the min and max frequency of 250 Hz and 1000 Hz.
For s=0.5, the sample is at the middle of the spectrum in logarithmic frequency space (500 Hz).

uniform sample: 0
    wavelength sample: [[0.3400000333786011]]
    frequency of sample: [[999.9998779296875]]
uniform sample: 0.5
    wavelength sample: [[0.6800000667572021]]
    frequency of sample: [[499.99993896484375]]
uniform sample: 1
    wavelength sample: [[1.3600001335144043]]
    frequency of sample: [[249.99996948242188]]
